In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from functools import reduce
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis

In [2]:
# Bureau Pred
b1_train = pd.read_csv('LeoPayu_BBL_Train_Pred.csv')
b1_eval= pd.read_csv('LeoPayu_BBL_Eval_Pred.csv')
b1_test = pd.read_csv('LeoPayu_BBL_Test_Pred.csv')
b1_oot = pd.read_csv('LeoPayu_BBL_Oot_Pred.csv')

In [3]:
#Pg Ped
pg_train = pd.read_csv('LeoPayu_PG_Train_Pred.csv')
pg_eval = pd.read_csv('LeoPayu_PG_Eval_Pred.csv')
pg_test= pd.read_csv('LeoPayu_PG_Test_Pred.csv')
pg_oot = pd.read_csv('LeoPayu_PG_Oot_Pred.csv')


In [4]:
#Make sure the size of records
b1_train.shape, pg_train.shape

((48952, 3), (48952, 3))

In [5]:
b1_test.shape, pg_test.shape

((40794, 3), (40794, 3))

In [6]:
b1_oot.shape, pg_oot.shape

((17343, 3), (17343, 3))

In [7]:
# Rename the pred columns
pg_train.drop(columns=['y_actual'],inplace=True)
b1_train.rename(columns={'y_pred':'BBL_pred'},inplace=True)
pg_train.rename(columns={'y_pred':'pg_pred'},inplace=True)

In [8]:
# Combine  all train pred
dfs = [b1_train,pg_train]
from functools import reduce
train= reduce(lambda left, right: pd.merge(left, right, on='LID'),dfs)

In [9]:
train.shape

(48952, 4)

In [10]:
train.columns

Index(['LID', 'y_actual', 'BBL_pred', 'pg_pred'], dtype='object')

In [11]:
# Rename the pred columns
pg_eval.drop(columns=['y_actual'],inplace=True)
b1_eval.rename(columns={'y_pred':'BBL_pred'},inplace=True)
pg_eval.rename(columns={'y_pred':'pg_pred'},inplace=True)
# Combine  all eval pred
dfs = [b1_eval,pg_eval]
eval_= reduce(lambda left, right: pd.merge(left, right, on='LID'),dfs)

In [12]:
eval_.columns

Index(['LID', 'y_actual', 'BBL_pred', 'pg_pred'], dtype='object')

In [13]:
# Rename the pred columns
pg_test.drop(columns=['y_actual'],inplace=True)
b1_test.rename(columns={'y_pred':'BBL_pred'},inplace=True)
pg_test.rename(columns={'y_pred':'pg_pred'},inplace=True)
# Combine  all test pred
dfs = [b1_test,pg_test]
test= reduce(lambda left, right: pd.merge(left, right, on='LID'),dfs)

In [14]:
dfs = [b1_test,pg_test]
test = reduce(lambda left, right: pd.merge(left, right, on='LID'),dfs)

In [15]:
test.shape

(40794, 4)

In [16]:
test.columns

Index(['LID', 'y_actual', 'BBL_pred', 'pg_pred'], dtype='object')

In [17]:
# Rename the pred columns
pg_oot.drop(columns=['y_actual'],inplace=True)
b1_oot.rename(columns={'y_pred':'BBL_pred'},inplace=True)
pg_oot.rename(columns={'y_pred':'pg_pred'},inplace=True)
# Combine  all oot pred
dfs = [b1_oot,pg_oot]
oot = reduce(lambda left, right: pd.merge(left, right, on='LID'),dfs)

In [18]:
oot.shape

(17343, 4)

In [19]:
oot.columns

Index(['LID', 'y_actual', 'BBL_pred', 'pg_pred'], dtype='object')

In [20]:
# Create Sqrt and cbrt from pred of each data
train['Bureau_Sqrt_pred'] = np.sqrt(train['BBL_pred'])
eval_['Bureau_Sqrt_pred'] = np.sqrt(train['BBL_pred'])
test['Bureau_Sqrt_pred'] = np.sqrt(test['BBL_pred'])
oot['Bureau_Sqrt_pred'] = np.sqrt(oot['BBL_pred'])


train['pg_Sqrt_pred'] = np.sqrt(train['pg_pred'])
eval_['pg_Sqrt_pred'] = np.sqrt(train['pg_pred'])
test['pg_Sqrt_pred'] = np.sqrt(test['pg_pred'])
oot['pg_Sqrt_pred'] = np.sqrt(oot['pg_pred'])

In [21]:
train['Bureau_cbrt_pred'] = np.cbrt(train['BBL_pred'])
eval_['Bureau_cbrt_pred'] = np.cbrt(train['BBL_pred'])
test['Bureau_cbrt_pred'] = np.cbrt(test['BBL_pred'])
oot['Bureau_cbrt_pred'] = np.cbrt(oot['BBL_pred'])

train['pg_cbrt_pred'] = np.cbrt(train['pg_pred'])
eval_['pg_cbrt_pred'] = np.cbrt(train['pg_pred'])
test['pg_cbrt_pred'] = np.cbrt(test['pg_pred'])
oot['pg_cbrt_pred'] = np.cbrt(oot['pg_pred'])

In [22]:
train

LID  y_actual  BBL_pred   pg_pred  Bureau_Sqrt_pred  pg_Sqrt_pred  \
0      18478462       0.0  0.190331  0.134944          0.436269      0.367347   
1      38033048       0.0  0.191878  0.119636          0.438038      0.345885   
2      34473140       1.0  0.361268  0.139204          0.601056      0.373101   
3      22941150       0.0  0.160979  0.139204          0.401221      0.373101   
4      35410355       0.0  0.248402  0.106307          0.498399      0.326048   
...         ...       ...       ...       ...               ...           ...   
48947  25030577       0.0  0.208475  0.142102          0.456591      0.376964   
48948  18029485       0.0  0.195295  0.114628          0.441922      0.338567   
48949  28857793       0.0  0.380483  0.115138          0.616833      0.339320   
48950  35478108       0.0  0.192592  0.119839          0.438853      0.346177   
48951  34120800       0.0  0.334483  0.139204          0.578345      0.373101   

       Bureau_cbrt_pred  pg_cbrt_pred  
0              0.575223      0.512922  
1              0.576777      0.492743  
2              0.712213      0.518264  
3              0.543988      0.518264  
4              0.628615      0.473719  
...                 ...           ...  
48947          0.592950      0.521835  
48948          0.580181      0.485769  
48949          0.724623      0.486489  
48950          0.577492      0.493021  
48951          0.694158      0.518264  

[48952 rows x 8 columns]

In [23]:
test

LID  y_actual  BBL_pred   pg_pred  Bureau_Sqrt_pred  pg_Sqrt_pred  \
0      30736127       1.0  0.539297  0.116615          0.734369      0.341489   
1      20926096       0.0  0.162433  0.135540          0.403030      0.368158   
2      37364110       0.0  0.319092  0.110050          0.564882      0.331738   
3      34138291       0.0  0.264340  0.139204          0.514140      0.373101   
4      25808943       0.0  0.187510  0.139204          0.433024      0.373101   
...         ...       ...       ...       ...               ...           ...   
40789  16305255       0.0  0.132077  0.104948          0.363424      0.323957   
40790  31415914       1.0  0.111232  0.139204          0.333515      0.373101   
40791  26883253       1.0  0.268885  0.121111          0.518542      0.348010   
40792  25033154       0.0  0.215808  0.115196          0.464551      0.339405   
40793  35025408       0.0  0.192929  0.139204          0.439237      0.373101   

       Bureau_cbrt_pred  pg_cbrt_pred  
0              0.813972      0.488560  
1              0.545621      0.513676  
2              0.683343      0.479214  
3              0.641782      0.518264  
4              0.572367      0.518264  
...                 ...           ...  
40789          0.509263      0.471692  
40790          0.480925      0.518264  
40791          0.645440      0.494760  
40792          0.599822      0.486571  
40793          0.577829      0.518264  

[40794 rows x 8 columns]

In [24]:
oot

LID  y_actual  BBL_pred   pg_pred  Bureau_Sqrt_pred  pg_Sqrt_pred  \
0      38061460       0.0  0.135571  0.134538          0.368200      0.366794   
1      38088287       0.0  0.175373  0.118822          0.418776      0.344706   
2      38063616       0.0  0.173043  0.123403          0.415984      0.351288   
3      38062570       0.0  0.226449  0.118513          0.475867      0.344258   
4      38372092       0.0  0.228392  0.117266          0.477904      0.342442   
...         ...       ...       ...       ...               ...           ...   
17338  40948851       0.0  0.304015  0.139204          0.551375      0.373101   
17339  40939643       0.0  0.413591  0.115510          0.643110      0.339868   
17340  40993256       0.0  0.260728  0.124388          0.510615      0.352687   
17341  41004875       0.0  0.141314  0.117262          0.375918      0.342435   
17342  41005451       1.0  0.492687  0.107956          0.701917      0.328567   

       Bureau_cbrt_pred  pg_cbrt_pred  
0              0.513716      0.512406  
1              0.559742      0.491624  
2              0.557252      0.497862  
3              0.609523      0.491197  
4              0.611261      0.489468  
...                 ...           ...  
17338          0.672406      0.518264  
17339          0.745058      0.487012  
17340          0.638845      0.499183  
17341          0.520869      0.489462  
17342          0.789812      0.476156  

[17343 rows x 8 columns]

In [25]:
X_train = train.drop(columns=['LID','y_actual'])
y_train = train['y_actual']

X_eval = eval_.drop(columns=['LID','y_actual'])
y_eval = eval_['y_actual']

X_test = test.drop(columns=['LID','y_actual'])
y_test = test['y_actual']

X_oot = oot.drop(columns=['LID','y_actual'])
y_oot = oot['y_actual']

In [26]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train, label=y_train)
deval = xgb.DMatrix(X_eval, label=y_eval)

dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [27]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 7.5,
    'alpha':5.5,
    'gamma':6.0,
    'random_state': 42
    
}

evals = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    early_stopping_rounds=30,
    evals=evals,
    verbose_eval=10
)

[0]	train-auc:0.67381	eval-auc:0.65067
[10]	train-auc:0.69571	eval-auc:0.67110
[20]	train-auc:0.69794	eval-auc:0.67234
[30]	train-auc:0.70011	eval-auc:0.67476
[40]	train-auc:0.70038	eval-auc:0.67495
[50]	train-auc:0.70080	eval-auc:0.67530
[60]	train-auc:0.70140	eval-auc:0.67584
[70]	train-auc:0.70162	eval-auc:0.67605
[80]	train-auc:0.70247	eval-auc:0.67689
[90]	train-auc:0.70318	eval-auc:0.67793
[99]	train-auc:0.70326	eval-auc:0.67808


In [28]:
# 4. **Predictions**
X_train['y_actual'] = y_train
X_train['y_pred'] = xgb_model.predict(dtrain)
X_train['LID'] = train.loc[X_train.index,'LID'].values

X_eval['y_actual'] = y_train
X_eval['y_pred'] = xgb_model.predict(deval)
X_eval['LID'] = eval_.loc[X_eval.index,'LID'].values

X_test['y_actual'] = y_test
X_test['y_pred'] = xgb_model.predict(dtest)
X_test['LID'] = test.loc[X_test.index,'LID'].values

X_oot['y_actual'] = y_oot
X_oot['y_pred'] = xgb_model.predict(doot)
X_oot['LID'] = oot.loc[X_oot.index,'LID'].values

In [29]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Train AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")

print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.70, Gini: 0.41, KS: 0.31
Train AUC: 0.64, Gini: 0.28, KS: 0.22
Test AUC: 0.68, Gini: 0.36, KS: 0.26
OOT AUC: 0.69, Gini: 0.39, KS: 0.29


In [30]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.277928         0.219286        0.280431        0.187114   4073   
9          0.162745         0.170653        0.186896        0.159164   4080   
8          0.133004         0.149196        0.159150        0.141841   4045   
7          0.125000         0.133355        0.141820        0.129278   3992   
6          0.097626         0.121327        0.129251        0.117066   4128   
5          0.089990         0.115925        0.117038        0.115086   4156   
4          0.066435         0.113021        0.115084        0.107913   2303   
3          0.066302         0.105445        0.107776        0.101705   5852   
2          0.051220         0.099015        0.101581        0.096233   2499   
1          0.041299         0.095347        0.096109        0.094662   5666   

        label_good  label_bad  
Decile                         
10            2941       1132  
9             3416        664  
8             3507        538  
7             3493        499  
6             3725        403  
5             3782        374  
4             2150        153  
3             5464        388  
2             2371        128  
1             5432        234

In [31]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.271676         0.224211        0.280431        0.190245   1730   
9          0.157956         0.173764        0.190177        0.161056   1722   
8          0.143516         0.150999        0.161054        0.142368   1735   
7          0.130410         0.133657        0.142319        0.128785   1733   
6          0.088356         0.121271        0.128698        0.117275   1709   
5          0.071220         0.115994        0.117267        0.114990   1713   
4          0.067416         0.112698        0.114962        0.107913   1068   
3          0.060692         0.105493        0.107776        0.102143   2455   
2          0.040573         0.099120        0.102125        0.096233   1257   
1          0.034219         0.095387        0.096109        0.094662   2221   

        label_good  label_bad  
Decile                         
10            1260        470  
9             1450        272  
8             1486        249  
7             1507        226  
6             1558        151  
5             1591        122  
4              996         72  
3             2306        149  
2             1206         51  
1             2145         76

In [32]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.289468         0.220351        0.280431        0.187120   4871   
9          0.170598         0.170931        0.187114        0.159656   4918   
8          0.134445         0.149522        0.159605        0.142073   4857   
7          0.132453         0.133642        0.142050        0.129278   4847   
6          0.088353         0.121325        0.129251        0.116714   4980   
5          0.080445         0.115865        0.116682        0.114990   4761   
4          0.066955         0.113056        0.114962        0.107913   2778   
3          0.052772         0.105432        0.107776        0.101581   7144   
2          0.038825         0.098954        0.101533        0.096233   2962   
1          0.034680         0.095340        0.096109        0.094662   6834   

        label_good  label_bad  
Decile                         
10            3461       1410  
9             4079        839  
8             4204        653  
7             4205        642  
6             4540        440  
5             4378        383  
4             2592        186  
3             6767        377  
2             2847        115  
1             6597        237

In [33]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

            Feature  Importance
0          BBL_pred       115.0
2  Bureau_Sqrt_pred        62.0
1           pg_pred        57.0
4  Bureau_cbrt_pred        38.0
3      pg_Sqrt_pred        18.0
5      pg_cbrt_pred         8.0
